# Basic Analysis
#### It's a little more than basic now, but *c'est la vie*
Now that we've cleaned up our data and have only the features we care about, we can run some basic statistical analysis to see if we can find any obvious patterns or interesting insights.

In [1]:
import pandas as pd
import numpy as np
import lzma, ast, gc
from scipy.sparse import lil_matrix, csr_matrix, save_npz, load_npz
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
with lzma.open("./cleaned_input/bills.pkl.xz", 'r') as f:
    bills = pd.read_pickle(f)
with lzma.open("./cleaned_input/people.pkl.xz", 'r') as f:
    people = pd.read_pickle(f)
with lzma.open("./cleaned_input/votes.pkl.xz", 'r') as f:
    votes = pd.read_pickle(f)

Let's take a quick look at our people dataframe, there are some interesting going on that might be interesting to point out.

In [3]:
people

,Name,Party,Role,State,District
ID,,,,,
6033,Carl Gatto,R,Rep,AK,HD-013
6034,Robert Lynn,R,Rep,AK,HD-026
6035,Max Gruenberg,D,Rep,AK,HD-016
6036,Nancy Dahlstrom,R,Rep,AK,HD-018
6037,Wes Keller,R,Rep,AK,HD-010
...,...,...,...,...,...
8675,Cale Case,R,Sen,WY,SD-025
8679,Dan Dockstader,R,Sen,WY,SD-016
8711,Dan Zwonitzer,R,Rep,WY,HD-043


Woah, 177,598 people have served in elected legislative positions since 2008? That seems wrong, I suspect there's probably a fair number of duplicates in there. Let's look at the dataframe with the duplicates removed.

In [4]:
people.loc[~people.duplicated()]

,Name,Party,Role,State,District
ID,,,,,
6033,Carl Gatto,R,Rep,AK,HD-013
6034,Robert Lynn,R,Rep,AK,HD-026
6035,Max Gruenberg,D,Rep,AK,HD-016
6036,Nancy Dahlstrom,R,Rep,AK,HD-018
6037,Wes Keller,R,Rep,AK,HD-010
...,...,...,...,...,...
24296,Forrest Chadwick,R,Rep,WY,HD-062
24307,Joshua Larson,R,Rep,WY,HD-017
24311,Stacy Jones,R,Sen,WY,SD-013


Much better, 21,761 is far more reasonable. It's important to note how we have removed duplicates, as we only removed rows that were exactly the same, representing people who served in the same position in multiple years. Some people have served in different positions or different districts, so it's important to keep those "duplicates," even though the indexes are the same because we do get some interesting information from that.

In [5]:
people = people.loc[~people.duplicated()]

Now we want to combine this with our people data so we can see how each person voted simply by looking at their information, rather than scraping through all votes. To do this, we'll collect all the votes of each person into a new dataframe then merge it with the current `people` dataframe.

In [6]:
try:
    with lzma.open("./cleaned_input/people_votes.pkl.xz", 'r') as f:
        people = pd.read_pickle(f)
except FileNotFoundError:
    people_votes = {}
    for roll_call in votes.index:
        vote = ast.literal_eval(votes.loc[roll_call]["Votes"])
        bill = votes.loc[roll_call]["Bill ID"]
        for person, v in vote:
            if person in people_votes.keys():
                people_votes[person].append((bill, roll_call, v))
            else:
                people_votes[person] = [(bill, roll_call, v)]
    for p in people_votes.keys():
        people_votes[p] = f"{people_votes[p]}"
    people_votes_df = pd.DataFrame.from_records(people_votes, index=["Votes"]).T
    people = people.join(people_votes_df)
    people.to_pickle("./cleaned_input/people_votes.pkl.xz")

Now that that's done, let's create a massive matrix of all people and all votes to make comparisons easy. This will take up a massive amount of space (18,000 x 1,292,603), but it will be insanely sparse, so we'll take advantage of that and use sparse matrix representation offered by SciPy.

In [7]:
try:
    smaller_votes = load_npz("./similarity_matrix_votes.npz")
except FileNotFoundError:
    people_votes_matrix = lil_matrix((np.max(people.index) + 1, np.max(votes.index) + 1), dtype="uint8")
    for person in people.index:
        if type(people.loc[person]["Votes"]) != str:
            if type(people.loc[person]["Votes"]) == float:
                continue
            if type(people.loc[person]["Votes"].iloc[0]) == float:
                continue
            person_votes = ast.literal_eval(people.loc[person]["Votes"].iloc[0])
        else:
            person_votes = ast.literal_eval(people.loc[person]["Votes"])
        for vote in person_votes:
            vote_id = vote[1]
            vote_actual = vote[2]
            people_votes_matrix[person, vote_id] = 1 if vote_actual == "Yea" else -1 if vote_actual == "Nay" else 0
    smaller_votes = csr_matrix(people_votes_matrix)
    save_npz("./similarity_matrix_votes.npz", smaller_votes)
    del people_votes_matrix
    gc.collect()

Alright, now let's run cosine similarity on this to determine similarity between representatives.

In [8]:
votes_similarities = cosine_similarity(smaller_votes)

Let's make sure that this is doing what we want it to do by checking to see who is similar to Lauren Boebert (I suspect MTG will be quite similar, as well as some other QAnon wackos).

In [60]:
boebert = people.loc[people["Name"] == "Lauren Boebert"].index[0]
people.loc[votes_similarities[boebert].argsort()[::-1][1:11]]

,Name,Party,Role,State,District,Votes
21935,Marjorie Greene,R,Rep,US,HD-GA-14,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21975,Robert Good,R,Rep,US,HD-VA-5,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21952,Matt Rosendale,R,Rep,US,HD-MT,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21934,Andrew Clyde,R,Rep,US,HD-GA-9,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21941,Mary Miller,R,Rep,US,HD-IL-15,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21969,Ronny Jackson,R,Rep,US,HD-TX-13,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21970,Troy Nehls,R,Rep,US,HD-TX-22,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21929,Byron Donalds,R,Rep,US,HD-FL-19,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21928,Kat Cammack,R,Rep,US,HD-FL-3,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
21967,Patrick Fallon,R,Rep,US,HD-TX-4,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."


Yeah, that all checks out. These people are very similar to Boebert, so it makes sense that they would all vote similarly. Looks like cosine similarity does exactly what we want it to do. 

Now let's add some similarity metrics to the bills so we can further compare representatives across states and legislative sessions.

In [10]:
with lzma.open("./cleaned_input/bills3.pkl.xz", 'r') as f:
    clustered_bills = pd.read_pickle(f)

In [11]:
with open("./cleaned_input/legiscam2.csv", 'r') as f:
    cluster_names = pd.read_csv(f)

In [12]:
clustered_bills

,Name,Number,State,Description,Status,Text,Progress,Sponsors,Votes,sentiment,Description2,topic
ID,,,,,,,,,,,,
1753037,Oceanside Academy volleyball champs,H4241,SC,recognize honor oceanside collegiate academy v...,Passed,,"[('2023-04-04', 1), ('2023-04-04', 4)]","[24305, 21741, 24124, 2202, 2205, 18085, 20147...",[],1,recognize honor oceanside collegiate academy v...,395
1604648,An Act Concerning Transit-oriented Development.,HB05429,CT,allow right development housing minimum overal...,Intro,https://legiscan.com/CT/text/HB05429/id/2542407,"[('2022-03-09', 1), ('2022-03-09', 9)]","[5759, 18265, 15086, 18262, 18281]",[],0,allow right development housing minimum overal...,2
1538680,Relating To Concessions.,HB1094,HI,provides department transportation flexibility...,Intro,https://legiscan.com/HI/text/HB1094/id/2455882,"[('2021-01-27', 1), ('2021-02-01', 9)]",[8342],[],0,provides department transportation flexibility...,-1
1698444,Bonds; authorize issuance to assist Petal Exce...,HB1557,MS,act authorize issuance state general obligatio...,Failed,https://legiscan.com/MS/text/HB1557/id/2686040,"[('2023-02-07', 1), ('2023-02-07', 9), ('2023-...",[6527],[],0,act authorize issuance state general obligatio...,147
1680034,"AN ACT to amend Tennessee Code Annotated, Titl...",SB0503,TN,"introduced, extends october november, time wit...",Intro,https://legiscan.com/TN/text/SB0503/id/2673085,"[('2023-01-25', 1)]",[21356],[],0,"introduced, extends october november, time wit...",130
...,...,...,...,...,...,...,...,...,...,...,...,...
1660327,Mississippi Rural Physicians Scholarship Resid...,SB2315,MS,"act amend section 37-144-3, mississippi code 1...",Failed,https://legiscan.com/MS/text/SB2315/id/2642210,"[('2023-01-16', 1), ('2023-01-16', 9), ('2023-...",[21460],[],0,"act amend section 37-144-3, mississippi code 1...",389
1729520,Minneapolis Minnesota Veterans Home historic b...,SF2731,MN,minneapolis minnesota veterans home historic b...,Intro,https://legiscan.com/MN/text/SF2731/id/2731308,"[('2023-03-08', 1), ('2023-03-08', 9)]",[6301],[],0,minneapolis minnesota veterans home historic b...,-1
1708728,INC TX-LONG TERM CARE,HB2598,IL,amends illinois income tax act. creates income...,Intro,https://legiscan.com/IL/text/HB2598/id/2700320,"[('2023-02-15', 1), ('2023-02-15', 9)]","[22579, 20901]",[],0,amends illinois income tax act. creates income...,166


It's very important to note that this is just a random sample of 20,000 bills from the past year, as running on a larger corpus was too intensive for the resources available to us. This may skew the results as we are missing quite a lot of data (1.8 million -> 20k, only 1/90 of the data).

In [13]:
cluster_names.set_index("Topic", inplace=True)
cluster_names.drop(["Unnamed: 0", "Count"], axis=1, inplace=True)

In [14]:
cluster_names

,Name
Topic,
-1,-1_amends_act_title
0,0_school_education_students
1,1_care_health_nursing
2,2_tax_property_income
3,3_election_elections_voter
...,...
406,406_elections_partisan_modifies
407,407_allergy_food_allergens
408,408_kidney_disease_optional


With our new shiny similarity metrics, we don't want to compare every single vote to every single other vote, primarily because that matrix would be MASSIVE (a dense 20,000 x 1,200,000 with each entry being a float, about 390,000,000gb). Instead, we'll average each representative's votes on each bill, then average all votes from all bills in each category to get a general "temperature" of how each person is voting on each topic. This is fairly rudamentary, and could definitely be improved in the future, but it works for now.

First, we'll convert the indexing of the votes dataframe to better see which votes pertain to each bill. This will allow us to go bill by bill, rather than vote by vote, making temperature calculations a bit easier.

In [15]:
votes = votes.reset_index().set_index(["Bill ID", "ID"])
votes

Description  Passed  \
Bill ID ID                                                                   
454312  306479                    Senate: <pre> SR 1 Final Passage    True   
472178  306480   Senate: CSHB 84(FIN)(efd am S) Third Reading -...    True   
        306481   Senate: CSHB 84(FIN)(efd am S) Third Reading -...    True   
        306482                                       House: Concur    True   
545632  306483                    House: Special Order of Business    True   
...                                                            ...     ...   
1673024 1268431                 Line Item Veto Override 27-3-1-0-0    True   
        1268432                 Line Item Veto Override 29-1-1-0-0    True   
        1268433                 Line Item Veto Override 27-3-1-0-0    True   
        1268434                 Line Item Veto Override 23-7-1-0-0    True   
1649186 1268518                    Adopted HB0093JC001: 27-0-4-0-0    True   

                                                             Votes  
Bill ID ID                                                          
454312  306479   [(6044, 'Yea'), (6061, 'Yea'), (6064, 'Yea'), ...  
472178  306480   [(6044, 'Yea'), (6061, 'Yea'), (6064, 'Yea'), ...  
        306481   [(6044, 'Yea'), (6061, 'Yea'), (6064, 'Yea'), ...  
        306482   [(6034, 'Yea'), (6035, 'Yea'), (6037, 'Yea'), ...  
545632  306483   [(6034, 'Yea'), (6035, 'Yea'), (6037, 'Yea'), ...  
...                                                            ...  
1673024 1268431  [(8641, 'Yea'), (8663, 'Yea'), (8675, 'Yea'), ...  
        1268432  [(8641, 'Yea'), (8663, 'Yea'), (8675, 'Yea'), ...  
        1268433  [(8641, 'Yea'), (8663, 'Yea'), (8675, 'Yea'), ...  
        1268434  [(8641, 'Nay'), (8663, 'Nay'), (8675, 'Yea'), ...  
1649186 1268518  [(8641, 'Yea'), (8663, 'Yea'), (8675, 'Yea'), ...  

[1292603 rows x 3 columns]

In [16]:
people_votes_temperatures = {}
for bill_id, vote_id in votes.index:
    # turn the string representation of the vote list into an actual list
    vote_list = ast.literal_eval(votes.loc[bill_id, vote_id]["Votes"])
    for vote in vote_list:
        # if we don't have this person yet, add them
        if vote[0] not in people_votes_temperatures:
            people_votes_temperatures[vote[0]] = {bill_id: [-1 if vote[1] == "Nay" else 1 if vote[1] == "Yea" else 0]}
        # if we have this person but don't have this bill for them, add it
        elif bill_id not in people_votes_temperatures[vote[0]]:
            people_votes_temperatures[vote[0]][bill_id] = [-1 if vote[1] == "Nay" else 1 if vote[1] == "Yea" else 0]
        # otherwise, add this vote to the end of the list
        else:
            people_votes_temperatures[vote[0]][bill_id].append(-1 if vote[1] == "Nay" else 1 if vote[1] == "Yea" else 0)
# average temperatures per bill
for person in people_votes_temperatures:
    for bill in people_votes_temperatures[person]:
        people_votes_temperatures[person][bill] = np.mean(people_votes_temperatures[person][bill])

Now we'll do something similar, but this time we'll incorporate the topics of each bill and give each person a temperature on a topic, rather than specific bills.

In [17]:
people_topics_temperature = {}
for person in people_votes_temperatures:
    for bill in people_votes_temperatures[person]:
        # if we don't have information for this bill, we'll skip it
        if bill not in clustered_bills.index:
            continue
        # grab topic and sentiment of this bill
        bill_topic = clustered_bills.loc[bill]["topic"]
        bill_sentiment = clustered_bills.loc[bill]["sentiment"]
        # if the sentiment is neutral, count as positive to not zero temperatures
        if bill_sentiment == 0:
            bill_sentiment = 1
        # if we don't have this person yet, add them
        # bill temperatures are multiplied by sentiment to normalize values
        if person not in people_topics_temperature:
            people_topics_temperature[person] = {bill_topic: [people_votes_temperatures[person][bill]] * bill_sentiment}
        # if we have this person but don't have this topic for them, add it
        elif bill_topic not in people_topics_temperature[person]:
            people_topics_temperature[person][bill_topic] = [people_votes_temperatures[person][bill] * bill_sentiment]
        # otherwise, add the temperature to the list
        else:
            people_topics_temperature[person][bill_topic].append(people_votes_temperatures[person][bill] * bill_sentiment)
# compute the mean temperature for each topic
for person in people_topics_temperature:
    for topic in people_topics_temperature[person]:
        people_topics_temperature[person][topic] = np.mean(people_topics_temperature[person][topic])

c:\Users\spwun\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\spwun\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Cool, now lets turn that into a big matrix so we can do the same kind of similarity measurements on it.

In [38]:
people_topics_matrix = lil_matrix((np.max(list(people_topics_temperature.keys())) + 1, np.max(cluster_names.index) + 1))
for person in people_topics_temperature:
    for topic in people_topics_temperature[person]:
        people_topics_matrix[person, topic] = people_topics_temperature[person][topic]
smaller_topics = csr_matrix(people_topics_matrix)
# remove NaN
smaller_topics.data = np.nan_to_num(smaller_topics.data)
save_npz("./similarity_matrix_topics.npz", smaller_topics)

In [39]:
topic_similarites = cosine_similarity(smaller_topics)

Now let's check to see if it did what we expected. Again, we'll look at Lauren Boebert to see who comes out as similar.

In [56]:
boebert = people.loc[people["Name"] == "Lauren Boebert"].index[0]
people.loc[topic_similarites[boebert].argsort()[::-1][0:10]]

,Name,Party,Role,State,District,Votes
16447,Rick Allen,R,Rep,US,HD-GA-12,"[(668287, 383926, 'Yea'), (668287, 383927, 'Ye..."
21944,Victoria Spartz,R,Rep,US,HD-IN-5,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
24055,Andrew Ogles,R,Rep,US,HD-TN-5,"[(1651107, 1231303, 'Nay'), (1651107, 1231304,..."
24053,Zachary Nunn,R,Rep,US,HD-IA-3,"[(1651107, 1231303, 'Nay'), (1651107, 1231304,..."
15243,Ann Wagner,R,Rep,US,HD-MO-2,"[(440461, 218634, 'Yea'), (440461, 218635, 'Na..."
24050,Nathaniel Moran,R,Rep,US,HD-TX-1,"[(1651107, 1231303, 'Nay'), (1651107, 1231304,..."
24048,Max Miller,R,Rep,US,HD-OH-7,"[(1651107, 1231303, 'Nay'), (1651107, 1231304,..."
24047,Marcus Molinaro,R,Rep,US,HD-NY-19,"[(1651107, 1231303, 'Nay'), (1651107, 1231304,..."
21945,Jacob Laturner,R,Rep,US,HD-KS-2,"[(1394633, 1014585, 'Yea'), (1460770, 1015299,..."
24060,George Santos,R,Rep,US,HD-NY-3,"[(1651107, 1231303, 'Nay'), (1651107, 1231304,..."


Interesting, this list is different from the earlier similarity list, even though it's still only people within the same legislative body. This is almost certainly a result of the reduced data size, and would be far more accurate with a larger corpus available to us.